In [70]:
import yfinance
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from pytorch_forecasting.models.temporal_fusion_transformer.sub_modules import GatedResidualNetwork
import gc
from tqdm.auto import tqdm
import wandb
import math
wandb.login()


True

In [3]:
Tickers = ["^GSPC", "^IXIC", "^TNX", "^SPGSCI", "^VIX", "VTI"]
Crypto = ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "USDT-USD"]
start="2012-01-01"
end="2022-12-31"

In [5]:
Stock_df = yfinance.download(Tickers, start=start, end=end, interval="1d")
Crypto_df = yfinance.download(Crypto, start=start, end=end, interval="1d")

[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  5 of 5 completed


In [6]:
def clean_stock(df,col):
    df = df.fillna(0)
    return df[col]
def clean_crypto(df,col):
    df = df.fillna(method='ffill').fillna(method='bfill')
    return df[col]

In [7]:
Stock_df = clean_stock(Stock_df,"Adj Close")
Crypto_df = clean_crypto(Crypto_df,"Adj Close")


/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/2017961215.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [8]:
Stock_df_pred = yfinance.download(Tickers, start=end,  interval="1d")
Crypto_df_pred = yfinance.download(Crypto, start=end,  interval="1d")

[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  5 of 5 completed


In [9]:
Stock_df_pred = clean_stock(Stock_df_pred,"Adj Close")
Crypto_df_pred = clean_crypto(Crypto_df_pred,"Adj Close")

/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/2017961215.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [10]:
before = Stock_df[:"2014-09-17"]
after = Stock_df["2014-09-17":]
after_concat = pd.concat([after,Crypto_df],axis=1).fillna(method='bfill')

/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/1571260081.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  after_concat = pd.concat([after,Crypto_df],axis=1).fillna(method='bfill')


In [11]:
test = pd.concat([Stock_df_pred,Crypto_df_pred],axis=1).fillna(method='bfill')

/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/1277879728.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test = pd.concat([Stock_df_pred,Crypto_df_pred],axis=1).fillna(method='bfill')


In [12]:
mu = before.rolling(1).mean()
mu.drop('^VIX',axis=1,inplace=True)
returns_tensor = torch.tensor(mu.values, dtype=torch.float32)

In [13]:
mu2 = after_concat.rolling(1).mean()
mu2.drop('^VIX',axis=1,inplace=True)
returns_tensor2 = torch.tensor(mu2.values, dtype=torch.float32)

In [14]:
mu_test = test.rolling(1).mean()
mu_test.drop('^VIX',axis=1,inplace=True)
returns_tensor_test = torch.tensor(mu_test.values, dtype=torch.float32)

In [26]:
class Time2Vector(nn.Module):
    def __init__(self, embed_dim):
        super(Time2Vector, self).__init__()
        self.embed_dim = embed_dim
    
    def forward(self, x):
        seq_length = x.size(1)
        positions = torch.arange(seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.embed_dim, 2).float() * (-math.log(10000.0) / self.embed_dim))
        pos_encoding = torch.zeros(seq_length, self.embed_dim)
        pos_encoding[:, 0::2] = torch.sin(positions * div_term)
        pos_encoding[:, 1::2] = torch.cos(positions * div_term)
        return pos_encoding


In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        print(in_features)
        print(out_features)
        self.w0 = nn.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.Parameter(torch.randn(1))
        self.w = nn.Parameter(torch.randn(in_features, out_features - 1))
        self.b = nn.Parameter(torch.randn(out_features - 1))
        print("shape w0",self.w0.shape)
        print("shape w",self.w.shape)
        print("shape b0",self.b0.shape)
        print("shape b",self.b.shape)
    def forward(self, tau, arg=None):
        if arg:
            v1 = torch.sin(F.linear(tau, self.w, self.b)) * arg
        else:
            v1 = torch.sin(F.linear(tau, self.w, self.b))

        v2 = F.linear(tau, self.w0, self.b0)

        return torch.cat([v1, v2], -1)


In [71]:

class Time2Vector(nn.Module):
    def __init__(self, seq_len):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

        # Linear part
        self.linear_weights = nn.Parameter(torch.Tensor(self.seq_len))
        self.linear_bias = nn.Parameter(torch.Tensor(self.seq_len))

        # Periodic part
        self.periodic_weights1 = nn.Parameter(torch.Tensor(self.seq_len))
        self.periodic_bias1 = nn.Parameter(torch.Tensor(self.seq_len))
        self.periodic_weights2 = nn.Parameter(torch.Tensor(self.seq_len))
        self.periodic_bias2 = nn.Parameter(torch.Tensor(self.seq_len))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.uniform_(self.linear_weights, -0.1, 0.1)
        nn.init.uniform_(self.linear_bias, -0.1, 0.1)
        nn.init.uniform_(self.periodic_weights1, -0.1, 0.1)
        nn.init.uniform_(self.periodic_bias1, -0.1, 0.1)
        nn.init.uniform_(self.periodic_weights2, -0.1, 0.1)
        nn.init.uniform_(self.periodic_bias2, -0.1, 0.1)

    def forward(self, x):
        # Mean reduction across the last dimension
        x = torch.mean(x, dim=-1)

        # Linear time feature
        time_linear = self.linear_weights * x + self.linear_bias
        time_linear = time_linear.unsqueeze(-1)

        # Periodic time features
        time_periodic1 = torch.sin(x * self.periodic_weights1 + self.periodic_bias1)
        time_periodic1 = time_periodic1.unsqueeze(-1)
        time_periodic2 = torch.sin(x * self.periodic_weights2 + self.periodic_bias2)
        time_periodic2 = time_periodic2.unsqueeze(-1)

        # Concatenate linear and periodic features
        return torch.cat([time_linear, time_periodic1, time_periodic2], dim=-1)
    

class EncoderLayer(nn.Module):
    """
    model2 = PortfolioTransformer(embed_dim=3027,asset_dim_in=11,asset_dim_out=10,nhead=8,num_model=256,batch_size=1)
    """
    def __init__(self, asset_dim_in, num_model,nhead, dropout=0.1,batch_size=1):  # Add input_dim parameter
        super(EncoderLayer, self).__init__()
        self.embedding = nn.Linear(asset_dim_in+3, num_model) #asset_dim_in = 11, num_model = 256
        self.multihead_attn = nn.MultiheadAttention(num_model, nhead)
        self.grn = GatedResidualNetwork(input_size=num_model,hidden_size=num_model,output_size=num_model)
        # self.grn = GatedResidualNetwork(d_model, dropout, batch_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(num_model)

    def forward(self, src, src_mask):
        src = self.embedding(src)
        residual = src
        src,_ = self.multihead_attn(src, src, src, attn_mask=src_mask)
        src = self.dropout(src)
        src = residual + src
        src = self.layer_norm(src)

        residual = src
        src = self.grn(src)
        src = self.dropout(src)
        src = residual + src
        src = self.layer_norm(src)

        return src
    
class DecoderLayer(nn.Module):
    def __init__(self, num_model, nhead, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.num_model = num_model
        self.masked_mha = nn.MultiheadAttention(num_model, nhead)
        self.encoder_attn = nn.MultiheadAttention(num_model, nhead, dropout=dropout)
        self.grn = GatedResidualNetwork(input_size=num_model,hidden_size=num_model,output_size=num_model)
        # self.dropout = nn.Dropout(dropout)
        self.layer_norm1 = nn.LayerNorm(num_model)
        self.layer_norm2 = nn.LayerNorm(num_model)

    def forward(self, tgt, memory, src_mask, tgt_mask):
        residual = tgt
        # Ensure tgt, memory, src_mask, and tgt_mask have the correct dimensions
        tgt = tgt.view(tgt.size(0), -1, self.num_model) # Reshape tgt to match the shape of memory
        # print("this is tgt shape {}".format(tgt.shape))
        memory = memory.view(memory.size(0), -1, self.num_model)
        # print("this is memory shape {}".format(memory.shape))
        tgt_mask = torch.ones(1,1)

        # Transpose tgt for masked_mha
        tgt = tgt.transpose(0, 1)

        # Masked multi-head attention within decoder
        tgt,_ = self.masked_mha(tgt, tgt, tgt, attn_mask=tgt_mask)
        # tgt = self.dropout(tgt)

        tgt = self.layer_norm1(residual + tgt)

        # Transpose tgt back to original order
        tgt = tgt.transpose(0, 1)
        # print(tgt.shape)
        # Multi-head attention to encoder output

        tgt= self.encoder_attn(tgt, tgt, memory, attn_mask=src_mask)
        # print(tgt[0].shape)
        # print(tgt[1].shape)
        # tgt = self.dropout(tgt)

        tgt = self.layer_norm2(residual + tgt[0])

        # Gated residual network
        tgt = self.grn(tgt)
        # tgt = self.dropout(tgt)
        tgt = self.layer_norm2(tgt + residual)

        return tgt
    
class PortfolioLoss(torch.nn.Module):
    def __init__(self, cost_rate, risk_free_rate=0.1):
        super(PortfolioLoss, self).__init__()
        self.cost_rate = cost_rate
        self.risk_free_rate = risk_free_rate

    def forward(self, weights, returns):
        # Ensure weights and returns are in the correct shape
        # Weights shape: [batch_size, num_assets]
        # Returns shape: [batch_size, num_assets]

        # Calculate portfolio returns
        # Element-wise multiplication and sum over assets for each instance in the batch
        portfolio_returns = torch.sum(weights * returns, dim=1)
        
        # Calculate portfolio volatility (standard deviation of returns)
        # You may need to adjust this calculation depending on how your returns are structured
        portfolio_volatility = torch.std(portfolio_returns, dim=0)

        # Calculate transaction costs
        # Assuming weights are adjacent allocations, calculate the change in weights
        deltas = torch.abs(weights[1:] - weights[:-1])
        transaction_costs = self.cost_rate * torch.sum(deltas, dim=1)

        # Calculate Sharpe Ratio
        # Note: risk_free_rate is typically a constant like a treasury bond yield
        sharpe_ratio = (portfolio_returns.mean() - self.risk_free_rate) / portfolio_volatility

        # Subtract transaction costs
        sharpe_ratio -= transaction_costs.mean()
        print("sharpe_ratio : ",sharpe_ratio)
        # Negate Sharpe Ratio to minimize this value during training
        cumulative_returns = np.cumprod(1 + returns) - 1
        peak, trough = np.argmax(np.maximum.accumulate(cumulative_returns) - cumulative_returns), np.argmax(cumulative_returns)
        max_drawdown = cumulative_returns[peak] - cumulative_returns[trough]
        average_return = portfolio_returns.mean()
        calmar = average_return / abs(max_drawdown)
        # return -sharpe_ratio , -calmar , -max_drawdown
        return -sharpe_ratio
    
class PortfolioTransformer(nn.Module):
    def __init__(self, embed_dim,asset_dim_in,asset_dim_out,nhead, num_model,batch_size=1):
        super(PortfolioTransformer, self).__init__()
        self.time2vec = Time2Vector(embed_dim)
        self.encoder = EncoderLayer(asset_dim_in=asset_dim_in,num_model=num_model,nhead=nhead,batch_size=batch_size)
        self.decoder = DecoderLayer(num_model=num_model,nhead=nhead)
        self.linear = nn.Linear(num_model, asset_dim_out)
        self.nhead = nhead

    def forward(self, x):
        
        time_features = self.time2vec(x)
        # print("X shape ",x.shape)
        x = torch.cat([x, time_features], dim=-1)

        src_mask = torch.ones(self.nhead, x.shape[0],x.shape[0], dtype=torch.float32)
        # print("X shape ",x.shape,"src_mask shape ",src_mask.shape)

        x = self.encoder(x, src_mask,)

        x = self.decoder(x, x, src_mask, src_mask)
        x = self.linear(x)
        x = x.mean(dim=1)
        # print("This is x before softmax",x)
        weights = torch.nn.functional.softmax(x, dim=-1)
        print("weight of asset shape",weights.shape)
        # print("weight of asset",weights)
        return weights

In [28]:
after_concat_tensor = torch.tensor(after_concat.values, dtype=torch.float32)
after_concat_tensor=after_concat_tensor[:374]
returns_tensor2 = returns_tensor2[:374]


In [29]:
test_tensor = torch.tensor(test.values, dtype=torch.float32)
test_tensor = test_tensor[:374]
returns_tensor_test = returns_tensor_test[:374]

# Start Training

In [72]:
config = dict(
    epochs=10,
    embed_dim=374,
    asset_dim_in=11,
    classes=10,
    cost_rate=0.01,
    batch_size=1,
    learning_rate=0.005,
    dataset="Price data",
    architecture="Transformer")

In [31]:
def make(config):

    # Make the model
    model = PortfolioTransformer(embed_dim=config.embed_dim,asset_dim_in=config.asset_dim_in,asset_dim_out=10,nhead=8,num_model=256,batch_size=1)

    # Make the loss and optimizer
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    
    return model, optimizer

In [32]:

def train(model, X, returns_tensor, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(X) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        loss = train_batch(X, returns_tensor, model, optimizer,config.cost_rate)
        example_ct +=  len(X)
        batch_ct += 1

        # Report metrics every 25th batch
        if ((batch_ct + 1) % 25) == 0:
            train_log(loss, example_ct, epoch)


def train_batch(X, returns_tensor, model, optimizer, cost_rate):
    X, returns_tensor = X, returns_tensor
    # Forward pass ➡
    weights = model(X)
    optimizer.zero_grad()
    loss = PortfolioLoss(cost_rate=cost_rate)(weights, returns_tensor2)
    # sharpe_loss, calmar_loss, max_dd_loss = PortfolioLoss(cost_rate=cost_rate)(weights, returns_tensor2)
    # total_loss = 0.7 * sharpe_loss + 0.2 * calmar_loss + 0.1 * max_dd_loss

    # total_loss.backward()
    loss.backward()
    optimizer.step()
    # return total_loss,sharpe_loss, calmar_loss, max_dd_loss
    gc.collect()
    return loss
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss" : loss})
    # wandb.log({"epoch": epoch, "total_loss": total_loss, "Sharpe_Loss" : sharpe_loss,"Calmar Loss" : calmar_loss, "Max DD Loss" : max_dd_loss }, step=example_ct)
    # print(f"Epoch {epoch + 1}/{100}, Total Loss: {total_loss.item():.4f}, Sharpe Loss: {sharpe_loss.item():.4f}, Calmar Loss: {calmar_loss.item():.4f}, Max DD Loss: {max_dd_loss.item():.4f}")
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [33]:
def model_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project="Portfolio Transformer", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, optimizer = make(config)
      # print(model)

      # and use them to train the model
      train(model, after_concat_tensor,returns_tensor2, optimizer, config)

      # and test its final performance
      test_model(model, test_tensor, returns_tensor_test)

    return model

In [34]:
def test_model(model, Y, returns_tensor_y):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        Y, returns_tensor_y = Y, returns_tensor_y
        weights = model(Y)
        loss= PortfolioLoss(cost_rate=0.01)(weights, returns_tensor_y)
        wandb.log({"test_loss": loss})


    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, Y, "model.onnx")
    wandb.save("model.onnx")

In [35]:
model = model_pipeline(config)

  0%|          | 0/10 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/4237996125.py", line 13, in model_pipeline
    train(model, after_concat_tensor,returns_tensor2, optimizer, config)
  File "/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/1913991516.py", line 10, in train
    loss = train_batch(X, returns_tensor, model, optimizer,config.cost_rate)
  File "/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/1913991516.py", line 22, in train_batch
    weights = model(X)
  File "/Users/veronicax/anaconda3/envs/is/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/Users/veronicax/anaconda3/envs/is/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1568, in _call_impl
    result = forward_call(*args, **kwargs)
  File "/var/folders/pc/gn4drv5d423127l84w13cgfr0000gn/T/ipykernel_25107/4262736333.py", line 174, in forward

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 374 but got size 11 for tensor number 1 in the list.

In [ ]:
#training model2
learning_rate = 0.001
cost_rate = 0.01
embed_dim = after_concat_tensor.shape[0]
asset_dim_in = after_concat_tensor.shape[1]
# Initialize model"asset_dim, embed_dim, nhead, num_encoder_layers"
model = PortfolioTransformer(embed_dim=embed_dim,asset_dim_in=asset_dim_in,asset_dim_out=10,nhead=8,num_model=256,batch_size=1)
# Define optimizer using weight from model1
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


#create checkpoint and callback
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./',
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min',
)
# Training loop and add checkpoint and callback
for epoch in range(100):

    weights = model(after_concat_tensor)
    # print("thsi is weight output".format(weights))
    # Calculate sharpe ratio loss
    # Backpropagate and update parameters
    optimizer.zero_grad()
    loss = PortfolioLoss(cost_rate=cost_rate)(weights, returns_tensor2)
    loss.backward()
    optimizer.step()



In [207]:
import torch
import torch.nn as nn

class Time2Vector(nn.Module):
    def __init__(self):
        super(Time2Vector, self).__init__()

        # Linear part
        self.linear_weights = nn.Parameter(torch.Tensor(1))
        self.linear_bias = nn.Parameter(torch.Tensor(1))

        # Periodic part
        self.periodic_weights1 = nn.Parameter(torch.Tensor(1))
        self.periodic_bias1 = nn.Parameter(torch.Tensor(1))
        self.periodic_weights2 = nn.Parameter(torch.Tensor(1))
        self.periodic_bias2 = nn.Parameter(torch.Tensor(1))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.uniform_(self.linear_weights, -0.1, 0.1)
        nn.init.uniform_(self.linear_bias, -0.1, 0.1)
        nn.init.uniform_(self.periodic_weights1, -0.1, 0.1)
        nn.init.uniform_(self.periodic_bias1, -0.1, 0.1)
        nn.init.uniform_(self.periodic_weights2, -0.1, 0.1)
        nn.init.uniform_(self.periodic_bias2, -0.1, 0.1)

    def forward(self, x):
        # Mean reduction across the last dimension
        x = torch.mean(x, dim=-1)

        seq_len = x.shape[-1]

        # Linear time feature
        time_linear = self.linear_weights * x + self.linear_bias
        time_linear = time_linear.unsqueeze(-1)

        # Periodic time features
        time_periodic1 = torch.sin(x * self.periodic_weights1 + self.periodic_bias1)
        time_periodic1 = time_periodic1.unsqueeze(-1)
        time_periodic2 = torch.sin(x * self.periodic_weights2 + self.periodic_bias2)
        time_periodic2 = time_periodic2.unsqueeze(-1)

        # Concatenate linear and periodic features
        return torch.cat([time_linear, time_periodic1, time_periodic2], dim=-1)

# Example usage
model = Time2Vector()

# Assuming x is your test set with shape (batch_size, sequence_length)
x_test = torch.rand((10, 600))

# Forward pass
output = model(x_test)
print(output.shape)


TypeError: Time2Vector.__init__() takes 1 positional argument but 2 were given